In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from os import listdir
from copy import deepcopy
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.backends.backend_pdf import PdfPages

rcParams = {'font.size': 17 , 'font.weight': 'normal', 'font.family': 'sans-serif',
            'axes.unicode_minus':False, 'axes.labelweight':'normal'}

graphspath = 'Mount-2/MetaCarvel_paper/hmp_scaffolds/stool/'
covpath = 'Research-Activities/Data/genomecov_d/'#'Mount-2/projects/refining_bins_with_assembly_graphs/genomecov_d/'
op_path = 'Scaffold_Coverage_After_Delinking/'

<h2>Computing Global Coordinates</h2>
<ol> 
    <li>This function computes the global coordinate system for the scaffold(connected component) based on the length of the contigs in the connected component and the linking information as returned by MetaCarvel. We order the contigs based on the topological sort of the subgraph and assign coordinates in the global coorinate system in breadth first manner.</li>   
    <li>We also consider the contig orientations and the edge orientations in accurate estimation of coordinates. </li>
    <li>If there multiple possible assignments we pick the one that has the largest coordinate value. We choose the greedy approaach because the number of solutions grows exponentially and the problem is NP-Hard! </li>
    <li>Finally, we normalize all other positions based on the least cooridnate value so that the coorinate system starts with 0.</li>
    <li>The input to the program is the connected component subgraph of the assembly graph ``oriented.gml" which is a result of running MetaCarvel.</li>
</ol>

In [2]:
def Compute_Global_Coordinates(subgraph):
    if not nx.is_directed_acyclic_graph(subgraph):
        return []
    top_sort = list(nx.topological_sort(subgraph))
    v_0 = top_sort[0]
    v_0_orientation = subgraph.nodes[v_0]['orientation']
    v_0_length = int(subgraph.nodes[v_0]['length'])
    if v_0_orientation == 'FOW':
        start = 0
        end = start + v_0_length
    elif v_0_orientation == 'REV':
        start = 0
        end = start-v_0_length
    global_coords = {v_0 : (start,end)}
    visited = dict(zip(top_sort, [False]*len(top_sort)))
    Q = [v_0]
    g_undir = subgraph.to_undirected()
    
    while len(Q) > 0:
        src = Q.pop()
        visited[src] = True
        neighbors = g_undir.neighbors(src)
        for n in neighbors:
            if subgraph.has_edge(src,n):
                ###Estimate n's coordinates based on the conting ordering src,n
                edge = (src, n)
                edge_orientation = subgraph.edges[edge]['orientation']
                edge_overlap = int(float(g_undir.edges[edge]['mean']))
                c2_length = int(g_undir.nodes[n]['length'])
                s1, e1 = global_coords[src]
                if edge_orientation == 'EE':
                    e2 = e1 + edge_overlap
                    s2 = e2 + c2_length
                if edge_orientation == 'EB':
                    s2 = e1 + edge_overlap
                    e2 = s2 + c2_length
                if edge_orientation == 'BB':
                    s2 = s1 + edge_overlap
                    e2 = s2 + c2_length
                if edge_orientation == 'BE':
                    e2 = s1 + edge_overlap
                    s2 = e2 + c2_length
                start,end = (s2, e2)
            else:
                ###Estimate n's coordinates based on the conting ordering n,src
                edge = (n,src)
                edge_orientation = subgraph.edges[edge]['orientation']
                edge_overlap = int(float(g_undir.edges[edge]['mean']))
                c1_length = int(g_undir.nodes[n]['length'])
                s2, e2 = global_coords[src]
                if edge_orientation == 'EE':
                    e1 = e2 - edge_overlap
                    s1 = e1 - c1_length
                if edge_orientation == 'EB':
                    e1 = s2 - edge_overlap
                    s1 = e1 - c1_length
                if edge_orientation == 'BB':
                    s1 = s2 - edge_overlap
                    e1 = s1 - c1_length
                if edge_orientation == 'BE':
                    s1 = e2 - edge_overlap
                    e1 = s1 - c1_length
                start,end = (s1,e1)
            try:
                if global_coords[n][0] < start:
                    global_coords[n] = (start,end)
            except KeyError:
                global_coords[n] = (start,end)
            if visited[n] == False:
                visited[n] = True
                Q.append(n)
    ###Normalize for the global coordinate system to start from 0
    min_coord = np.inf
    for g in global_coords:
        start,end = global_coords[g]
        min_val = min(start,end)
        if min_val < min_coord:
            min_coord = min_val
    for g in global_coords:
        s,e = global_coords[g]
        global_coords[g] = s-min_coord, e-min_coord
    return global_coords

<h2> Computing Coverages </h2>
    
To compute the read coverages we use the *genomecov* program, part of the *bedtools* suite. We run *genomecov* with *-d* optional enabled so that we get the per base depth. The output of the prgram is utilized to compute the depth along the scaffold and this function loads the out of genomecov as a dataframe.   

In [3]:
def Load_Read_Coverage(sample):
    df_coverage = pd.read_csv(covpath+sample+'.txt',names = ['Contig','Loc','coverage'], 
                              sep = '\t', low_memory = False,  
                              dtype = {'Contig': str, 'Loc': np.int32, 'Coverage': np.int32},
                              index_col = 'Contig', engine='c')
    df_coverage['Loc'] = df_coverage['Loc']-1
    return df_coverage

<h2> Computing the Depth of Coverage along the Scaffold's Global Coordinate System  </h2>

To compute the depth we run the *Compute_Global_Coordinates* and *Load_Read_Coverage* functions and pass this to the *Compute_Coverage* described below. This program estimates the per cooridnate depth by using the outputs of the said two information. The program returns the coverage along the entire scaffold and the coverage along the longest path and a dataframe contiaining the contig, its average depth and its coordinates in the scaffold.   

In [4]:
def Compute_Coverage(connected_component, df_coverage):
    top_sort = list(nx.topological_sort(connected_component))
    coords = Compute_Global_Coordinates(connected_component)
    max_coord = -np.inf
    for c in top_sort:
        max_v = max(coords[c])
        if max_v >= max_coord: max_coord = max_v
    coverage = np.zeros(max_coord+1)
    d_arr = []
    for c in top_sort:
        contig_depth = np.array(df_coverage.loc[c]['coverage'])
        d = {'Contig':c, 'Average_Depth':np.median(contig_depth),'Coords':coords[c]}
        d_arr.append(d)
        s,e = coords[c]
        if (s > e): coverage[s:e:-1] += contig_depth[::-1]
        else: coverage[s:e] += contig_depth
    df_depths = pd.DataFrame(d_arr)
    df_depths = df_depths.set_index('Contig')
    df_depths = df_depths.loc[top_sort]
    longest_coverage = np.zeros(max_coord+1)
    longest_path = list(nx.dag_longest_path(connected_component))
    for c in longest_path:
        contig_depth = np.array(df_coverage.loc[c]['coverage'])
        s,e = coords[c]
        if (s > e): longest_coverage[s:e:-1] += contig_depth[::-1]
        else: longest_coverage[s:e] += contig_depth
    return coverage, longest_coverage, df_depths

<h2> Change Point Detection on the Scaffold Coverage </h2>

To compute changepoints along the scaffold we slide a window of default size 1500 along the scaffold and take the ratios of the maximum of means of the window preceeding any point and the window suceeding the point and the minimum of the said quantities. Any abnromal spike magnitude indicates a change point. If the size of the contig is less than the size of the window we adaptively pick a smaller window. 

In [5]:
def Helper_Changepoints(cov_vec, window_size = 1500):
    indices_non_zero = np.where(cov_vec > 0)
    sliced = cov_vec[indices_non_zero]
    while window_size >= len(sliced)/2:
        window_size = int(window_size/5)
        print(window_size)
    m = []
    for i in range(window_size, len(sliced)-window_size):
        pred = sliced[i-window_size:i].mean()
        succ = sliced[i:i+window_size].mean()
        if pred == 0 or succ == 0: 
            r = 0
        else: 
            r = max(pred,succ)/min(pred,succ)
        m.append(r)
        
    mean_ratios = [0]*window_size + m + [0]*window_size
    cpts = np.zeros(len(cov_vec))
    cpts[indices_non_zero] = mean_ratios
    return cpts

<h2> Outlier Detection in Change Point Signals </h2>
<ol>
    <li>The following code segment is used to identify outliers in change points. To do so, we compute the peaks first.</li> 
    <li>We don't directly identify outliers on the signal because doing so would pickup points near the peaks, these are indicators of sliding windows and not really outliers.</li> 
    <li>To overcome the issue, we first identify peaks. A point is a peak if a point is larger than its predecessor and successor. This is performed by the function *ID_Peaks*.</li> 
    <li>The output of this passed to *ID_outliers* which picks all those points that is gretaer than the point which is the *thresh*'s percentile. The default value of *thresh* is 98.5. </li>
    <li>The filter outliers is still a work in progress. This is aimed at removing all the outliers points that is close to one another, but this is not super important. While the mthod described in the following block is data driven we are working on improving this method by examining the underlying graph structure.</li>
</ol>

In [6]:
def ID_outliers(change_point_vec, thresh):
    cutoff = np.percentile(change_point_vec, thresh)
    indices = np.where(change_point_vec >= cutoff)
    return indices

def ID_Peaks(change_point_vec, thresh = 98.5):
    indices = range(0, len(change_point_vec))
    Peak_Indices = []
    for i in range(1,len(indices)-1):
        prev_index, curr_index, next_index = indices[i-1], indices[i], indices[i+1]
        pt1, pt2, pt3 = change_point_vec[prev_index], change_point_vec[curr_index], change_point_vec[next_index]
        if max(pt1, pt2, pt3) == pt2 and pt2 > 0: Peak_Indices.append(curr_index)
    Peak_Indices = np.array(Peak_Indices)
    Peak_Values = change_point_vec[Peak_Indices]
    outlier_peaks = ID_outliers(Peak_Values, thresh)
    return Peak_Indices[outlier_peaks]#filtered_outliers

def Filter_Neighbors(outlier_list, changepoints, window_size = 100):
    if len(outlier_list) == 0:
        print(outlier_list)
        return outlier_list
    filtered_outlier_list = []
    i = 0
    cmp = outlier_list[0]
    outlier_set = set({cmp})
    for i in range(1, len(outlier_list)):
        curr = outlier_list[i]
        if (curr - cmp) <= window_size:
            if changepoints[curr] >= changepoints[cmp]:
                outlier_set.remove(cmp)
                outlier_set.update([curr])
                cmp = curr
            else:
                outlier_set.update([cmp])
        else:
            outlier_set.update([curr])
            cmp = curr
    outlier_set = list(outlier_set)
    outlier_set.sort()
    return outlier_set

<h2> Rules for delinking contigs at change points. </h2> 

The rules for delinking the contigs are described below. The first row represent the orientation of the contig and the first colum represent the location of the contig relative to the contig at the change point. 

|#                   |  Forward            |  Reverse          |
|:------------------:|---------------------|-------------------|
|        Start       | Delink predecessor  | Delink successor  |
|          End       | Delink successor    | Delink predecessor|

The input to the program is the set of outliers, the dictionary containing the contigs at each position, a vector of means for detecting change points and a *pos_cutoff* that specifies the number of basepairs from the contig ends to considered for delinking.

In [7]:
def Get_Outlier_Contigs(outliers, positions, coordinates, graph, pos_cutoff = 100):
    g_ = deepcopy(graph)
    potential_contigs_removal = {}
    for o in outliers:
        contigs_intersecting = positions[o]
        closest_contig, closest_contig_val = '',np.inf
        forward, start = True, True
        for contig in contigs_intersecting:
            s,e = coordinates[contig]
            if s>e: f = False
            else: f = True
                
            if np.abs(s-o) < closest_contig_val:
                closest_contig_val= np.abs(s-o) 
                closest_contig = contig
                forward = f
            if np.abs(e-o) < closest_contig_val:
                closest_contig_val= np.abs(e-o) 
                closest_contig = contig
                forward = f
                start = False
        if closest_contig_val <= pos_cutoff:
            potential_contigs_removal[closest_contig] = (closest_contig_val, o, forward, start)
    
    for c in potential_contigs_removal:
        p = potential_contigs_removal[c]
        #print(c,p)
        fwd, start = p[2], p[3]       
        if(fwd == True) and (start == True): 
            print('Removing',c,'\'s Predecessors')
            predecessors = list(g_.predecessors(str(c)))
            if len(predecessors) > 0:
                edges = [(pred,c) for pred in predecessors]
                g_.remove_edges_from(edges)
        if(fwd == False) and (start == True): 
            print('Removing',c,'\'s Successors')
            successors = list(g_.successors(str(c)))
            if len(successors) > 0:
                edges = [(c,succ) for succ in successors]
                g_.remove_edges_from(edges)
        if(fwd == False) and (start == False): 
            print('Removing',c,'\'s Predecessors')
            predecessors = list(g_.predecessors(str(c)))
            if len(predecessors) > 0:
                edges = [(pred,c) for pred in predecessors]
                g_.remove_edges_from(edges)
        if(fwd == True) and (start == False): 
            print('Removing',c,'\'s Successors')
            successors = list(g_.successors(str(c)))
            if len(successors) > 0:
                edges = [(c,succ) for succ in successors]
                g_.remove_edges_from(edges)
    return g_
         

<h2> Visualize the Results </h2>

In this code fragment we plot two figures, one describing 
<ol>
    <li>The Pileup of Contigs</li> 
    <li>Coverages along the global Coordinate </li> 
    <li>Change Point Metric </li> 
    <li>The Coverages of the Various Contigs in the Scaffold</li> 
    The program calls the methods above to perform the computations. 
</ol>

The second figure plots the coverage of a contig with respect to its predecessors and successors. 

In [31]:
def Return_Contig_Scaffold_Positions(coordinate_dictionary):
    pos_dict = {}
    for c in coordinate_dictionary:
        start, end = coordinate_dictionary[c]
        if start > end: a,b = end, start
        else: a,b = start,end
        for i in range(a, b+1):
            try: pos_dict[i].append(c)
            except KeyError: pos_dict[i] = [c]
    return pos_dict


def Write_Coverage_Outputs(graph,df_coverage):
    weakly_connected_components = list(nx.weakly_connected_component_subgraphs(graph))
    cc_ctr = 0
    list_coverages = []
    list_coords = []
    for i in range(len(weakly_connected_components)):
        test = weakly_connected_components[i]
        if nx.is_directed_acyclic_graph(test):
            print(i, len(test.nodes()))
            nodes = list(test.nodes())
            df_coverage_cc = df_coverage.loc[nodes]
            coverage, longest_coverage, df_depths = Compute_Coverage(test, df_coverage_cc)
            mean_ratios = Helper_Changepoints(deepcopy(coverage))
            outliers = ID_Peaks(mean_ratios)
            outliers = Filter_Neighbors(outliers, mean_ratios)
    
            top_sort = df_depths.index.tolist()
            coords = dict(zip(df_depths.index.tolist(), df_depths['Coords'].tolist()))
            Pos_Dict = Return_Contig_Scaffold_Positions(coords)
            df_depths = df_depths.loc[top_sort]

            g_removed = Get_Outlier_Contigs(outliers, Pos_Dict, coords, test, 100)
            delinked_conn_comps = list(nx.weakly_connected_component_subgraphs(g_removed))
            
            for cc in delinked_conn_comps:
                cc_ctr += 1
                coverage_cc, longest_coverage_cc, df_depths_cc = Compute_Coverage(cc, df_coverage_cc)
                coords_cc = dict(zip(df_depths_cc.index.tolist(), df_depths_cc['Coords'].tolist()))
                for i in range(len(coverage_cc)):
                    d = {'Connected_Component':cc_ctr, 'Coordinates':i,'Coverage':coverage_cc[i]}
                    list_coverages.append(d)
                for c in coords_cc:
                    d = {'Connected_Component':cc_ctr, 'Contig':c, 
                         'Start':coords_cc[c][0],'End':coords_cc[c][1]}
                    list_coords.append(d)
        print('\n')
    df_coords = pd.DataFrame(list_coords)
    df_coords = df_coords.sort_values(by = ['Connected_Component'])
    
    df_coverages_scaffolds = pd.DataFrame(list_coverages)
    df_coverages_scaffolds = df_coverages_scaffolds.sort_values(by = ['Connected_Component','Coordinates'])
    
    return df_coords, df_coverages_scaffolds

def Draw_Plots(connected_component, title, df_coverage):
    coverage, longest_coverage, df_depths = Compute_Coverage(connected_component, df_coverage)
    mean_ratios = Helper_Changepoints(deepcopy(coverage))
    outliers = ID_Peaks(mean_ratios)
    outliers = Filter_Neighbors(outliers, mean_ratios)
    
    top_sort = df_depths.index.tolist()
    coords = dict(zip(df_depths.index.tolist(), df_depths['Coords'].tolist()))
    Pos_Dict = Return_Contig_Scaffold_Positions(coords)
    df_depths = df_depths.loc[top_sort]
    
    contigs_set_outliers = set({})
    for o in outliers:
        contigs = Pos_Dict[o]
        contigs_set_outliers.update(contigs)
    contigs_set_outliers = list(contigs_set_outliers)
    
    fig_line_chart, ax_line_chart = plt.subplots(4, 1, figsize=(20, 40),
                    gridspec_kw={'height_ratios': [2.75 ,0.75, 0.75, 0.75]})
    ctr, ctr_inc  = 0.5, 0.5
    yticks, ytick_labels = [ctr], []
    for k in top_sort:
        start,end = coords[k]
        if start > end: color = 'red'
        else: color = 'blue'
        if k in contigs_set_outliers: color = 'green'
        ax_line_chart[0].plot([start, end],[ctr, ctr], linewidth = 3, color = color)
        ax_line_chart[0].axhline(ctr, color = 'black', linewidth=0.5)
        ctr += ctr_inc
        yticks.append(ctr)
        ytick_labels.append(k)
    ax_line_chart[0].set_yticks(yticks)
    colors = ['red', 'blue', 'green']
    lines = [Line2D([0], [0], color=c, linewidth=3) for c in colors]
    labels = ['Reverse', 'Forward','Outlier']
    ax_line_chart[0].legend(lines, labels)
    ax_line_chart[0].set_yticklabels(ytick_labels)
    #ax_line_chart[0].set_xlim([14000, 22000])
    
    ax_line_chart[1].plot(coverage, color = 'teal', label = 'Scaffold Coverage')
    ax_line_chart[1].plot(longest_coverage, color = 'orange', label = 'Longest Path Coverage')
    ax_line_chart[1].set_xlabel('Scaffold Coordinate')
    ax_line_chart[1].set_ylabel('Fold Coverage')
    ax_line_chart[1].legend()
    
    ax_line_chart[2].plot(mean_ratios, color = 'brown', linewidth = 2)
    ax_line_chart[2].set_ylabel('Change Points')
    for o in outliers:
        ax_line_chart[0].axvline(o, color = 'black', linewidth = 0.8, linestyle = '--')
        ax_line_chart[1].axvline(o, color = 'black', linewidth = 0.8, linestyle = '--')
        ax_line_chart[2].axvline(o, color = 'black', linewidth = 0.8, linestyle = '--')
    outliers.sort()
    temp = [0]+list(outliers)+[len(coverage)]
    for i in range(len(temp)-1): 
        if i%2 == 0: c = 'pink'
        else: c = 'black'
        ax_line_chart[0].axvspan(temp[i], temp[i+1], alpha=0.3, color=c)
        ax_line_chart[1].axvspan(temp[i], temp[i+1], alpha=0.3, color=c)
        ax_line_chart[2].axvspan(temp[i], temp[i+1], alpha=0.3, color=c)

    outlier_bars = []
    for i in range(0, len(top_sort)):
        if top_sort[i] in contigs_set_outliers:
            outlier_bars.append(i)
            
    bars = ax_line_chart[3].bar(df_depths.index, df_depths['Average_Depth'], color = 'green')
    ax_line_chart[3].set_xticklabels(df_depths.index, rotation=90)#, ha='right')
    for b in outlier_bars:
         bars[b].set_color('r')
    ax_line_chart[3].set_ylabel('Median Coverage')
    fig_line_chart.suptitle(title)
    fig_line_chart.tight_layout()
    fig_line_chart.subplots_adjust(top=0.97, left = 0.15)
    
    fig_depth_scatter, ax_depth_scatter = plt.subplots(2,1,figsize=(20,12), gridspec_kw={'height_ratios': [2,1]})
    pos, depth_ratio = 0, []
    for c in coords:
        depths_pred = list(df_depths.loc[list(connected_component.predecessors(c))]['Average_Depth'])
        depths_succ = list(df_depths.loc[list(connected_component.successors(c))]['Average_Depth'])
        x_pred = [pos for i in range(len(depths_pred))]
        x_succ = [pos for i in range(len(depths_succ))]
        ax_depth_scatter[0].scatter(x_pred, depths_pred, color = 'blue', marker = 'x')
        ax_depth_scatter[0].scatter(x_succ, depths_succ, color = 'olive', marker = '<')
        depth = df_depths.loc[c]['Average_Depth']
        depths = depths_pred+depths_succ
        depth_ratio.append(depth/max(depths))
        if max(depths) < depth and len(depths) > 2:
            ax_depth_scatter[0].scatter(pos, depth, color = 'black', s = 150, facecolors='none', linewidth = 3)
        ax_depth_scatter[0].scatter(pos, depth, color = 'red', s = 80, alpha = 0.6)
        ax_depth_scatter[0].axvline(pos, linewidth = 0.3, linestyle = '--', color = 'black')
        ax_depth_scatter[1].axvline(pos, linewidth = 0.3, linestyle = '--', color = 'black')
        pos += 1
    ax_depth_scatter[0].set_xticks([])
    
    ax_depth_scatter[1].plot(range(0, len(top_sort)), depth_ratio, marker = 'o', color = 'blue')
    ax_depth_scatter[1].set_xticks(range(0, len(top_sort)))
    ax_depth_scatter[1].set_xticklabels(top_sort, rotation = 90)
    ax_depth_scatter[1].axhline(1, color = 'black')
    
    fig_depth_scatter.suptitle(title)
    fig_depth_scatter.tight_layout()
    fig_depth_scatter.subplots_adjust(top=0.94, bottom = 0.25)
    
    g_removed = Get_Outlier_Contigs(outliers, Pos_Dict, coords, connected_component, 100)
    conns = list(nx.weakly_connected_component_subgraphs(g_removed))
    fig_removed, ax_removed = plt.subplots(len(conns),1, figsize = (20,40), 
                                          gridspec_kw={'height_ratios': [1.0/len(conns)]*len(conns)})

    for i in range(len(conns)):
        conn = conns[i]
        coverage, longest_coverage, df_depths = Compute_Coverage(conn, df_coverage)
        if len(conns) > 1:
            ax_removed[i].plot(coverage, color = 'red',label = 'Scaffold coverage')
            ax_removed[i].plot(longest_coverage, color = 'blue', label = 'Longest Path Coverage')
            ax_removed[i].legend()
            ax_removed[i].set_title('Number of Contigs:'+str(len(conn)))
        else:
            ax_removed.plot(coverage, color = 'red',label = 'Scaffold coverage')
            ax_removed.plot(longest_coverage, color = 'blue', label = 'Longest Path Coverage')
            ax_removed.legend()
            ax_removed.set_title('Number of Contigs:'+str(len(conn)))
    fig_removed.tight_layout()
    fig_removed.suptitle(title)
    fig_removed.subplots_adjust(top=0.94)
    
    return fig_line_chart, fig_depth_scatter, fig_removed

In [32]:
samples = listdir(covpath)
samples.sort()
plt.rcParams.update(rcParams)
#samples = ['SRS019397']

This part of the code just calls the "Draw_Plots" codes. This calls the various above methods. These are the various variables to look out for, 
<ol>
    <li>coverage = Scaffold level coverage</li>
    <li>longest_coverage = Coverage along the longest path</li>
    <li>mean_ratios = The ratios used to detect change points </li>
    <li>outliers = Outliers detected on the change point signal </li>
    <li>coords = The dictionary with the contigs as the keys and coordinates as the value</li>
    <li>Pos_Dict = This dictionary gives the contig ids at every position along the scaffold</li>
</ol>

Return whichever variable you want for your analysis.

If you want to run for a particular sample set that to the sample, you dont have to iterate through all the samples. 

Similarly just mention the connected component id you are interested in viewing the plot for the second loop. For an example the connected component we are looking at now is connected component id 16 in my machine. This could be different in various machines. We are looking at a scaffold with 73 contigs and the sample id is 'SRS012902.txt'. 


In [ ]:
for s in samples[6:]:
    sample = s.replace(".txt","")
    if sample[0] == '.':
        continue
    print(s)
    Graph_path= graphspath+sample+'/'+sample+'_scaffolds/oriented.gml'
    G = nx.read_gml(Graph_path)
    weakly_connected_components = list(nx.weakly_connected_component_subgraphs(G))
    df_coverage =Load_Read_Coverage(sample) 
    
    pdf_1 = PdfPages(op_path+sample+'.pdf')
    
    for i in range(len(weakly_connected_components)):
        test = weakly_connected_components[i]
        if len(test) >= 3 and nx.is_directed_acyclic_graph(test):
            nodes = list(test.nodes())
            coverage = df_coverage.loc[nodes]
            fig1, fig2, fig3 = Draw_Plots(test, 'Connected Component:'+str(i), coverage)
            pdf_1.savefig( fig1 )
            pdf_1.savefig( fig2 )
            pdf_1.savefig( fig3 )
            plt.close('all')
            print(i, len(weakly_connected_components))
            
    pdf_1.close()

SRS013215.txt
1 692
Removing 1003 's Successors
2 692
Removing 2280 's Successors
Removing 237 's Successors
4 692
Removing 75 's Predecessors
Removing 777 's Successors
6 692
Removing 823 's Predecessors
Removing 1016 's Successors
Removing 472 's Successors
8 692
9 692
Removing 1483 's Successors
Removing 1688 's Predecessors
Removing contig-100_5043.5043 's Predecessors
Removing 1018 's Predecessors
10 692
Removing 1019 's Predecessors
11 692
12 692
Removing 153 's Predecessors
Removing 1063 's Predecessors
Removing contig-100_1989.19623 's Successors
Removing 457 's Predecessors
Removing 1029 's Successors
Removing contig-100_2606.20240 's Successors
Removing 161 's Successors
15 692
Removing 820 's Successors
18 692
Removing 1108 's Predecessors
Removing contig-100_5452.5452 's Predecessors
Removing contig-100_680.18314 's Successors
Removing contig-100_481.18115 's Successors
20 692
Removing 1134 's Predecessors
Removing contig-100_3235.20869 's Successors
21 692
Removing 1044 's

5 1342
7 1342
Removing 5386 's Successors
Removing 932 's Successors
Removing 6323 's Successors
8 1342
Removing contig-100_10283.39600 's Predecessors
Removing 315 's Successors
9 1342
Removing 647 's Predecessors
Removing 6306 's Predecessors
Removing contig-100_10941.40258 's Successors
Removing 5921 's Successors
Removing 1036 's Successors
Removing 6517 's Successors
10 1342
Removing 2728 's Successors
Removing 1038 's Successors
11 1342
Removing 5693 's Successors
Removing 1655 's Successors
12 1342
Removing contig-100_1991.50948 's Predecessors
15 1342
16 1342
Removing 5571 's Successors
17 1342
18 1342
Removing 5945 's Predecessors
Removing 5962 's Successors
19 1342
Removing 1821 's Predecessors
Removing 1234 's Successors
Removing contig-100_1982.31299 's Successors
20 1342
21 1342
Removing 1623 's Successors
Removing 1061 's Successors
22 1342
Removing 3466 's Successors
Removing 1065 's Predecessors
23 1342
Removing 1074 's Predecessors
25 1342
Removing 1078 's Successors
R

317 1342
319 1342
Removing 7040 's Successors
322 1342
323 1342
324 1342
Removing 5592 's Successors
326 1342
327 1342
328 1342
Removing 2481 's Successors
Removing 6974 's Successors
330 1342
331 1342
Removing 5805 's Predecessors
Removing contig-100_2265.51222 's Successors
333 1342
335 1342
Removing contig-100_1968.31285 's Predecessors
339 1342
Removing 6599 's Predecessors
340 1342
343 1342
344 1342
345 1342
Removing 3402 's Predecessors
Removing 6748 's Predecessors
Removing contig-100_1176.54509 's Successors
357 1342
358 1342
Removing contig-100_3509.52466 's Predecessors
Removing 5885 's Successors
359 1342
Removing 5537 's Successors
360 1342
Removing contig-100_1998.50955 's Successors
361 1342
Removing 5652 's Predecessors
Removing 2319 's Predecessors
Removing 481 's Successors
Removing 3444 's Predecessors
Removing 3690 's Successors
364 1342
Removing 6142 's Predecessors
366 1342
367 1342
369 1342
Removing contig-100_9987.39304 's Successors
Removing contig-100_776.776 '

65 3738
66 3738
68 3738
Removing 12854 's Predecessors
Removing contig-100_45.80469 's Successors
Removing 1808 's Successors
69 3738
Removing 10126 's Successors
70 3738
Removing 11055 's Successors
71 3738
74 3738
Removing contig-100_8987.64046 's Predecessors
75 3738
78 3738
Removing 10902 's Predecessors
Removing 8280 's Successors
85 3738
Removing contig-100_4009.76253 's Successors
87 3738
88 3738
89 3738
Removing 7564 's Successors
90 3738
Removing contig-100_1291.78164 's Predecessors
Removing 11353 's Predecessors
Removing 2891 's Successors
Removing contig-100_1432.84818 's Successors
92 3738
Removing 10169 's Predecessors
Removing 5299 's Successors
Removing 12910 's Successors
94 3738
Removing contig-100_2289.88654 's Successors
97 3738
98 3738
Removing 1018 's Successors
99 3738
Removing 2541 's Predecessors
100 3738
101 3738
Removing 6388 's Successors
102 3738
Removing 73 's Successors
108 3738
Removing 13564 's Successors
Removing contig-100_384.86749 's Successors
109 

552 3738
555 3738
Removing 12270 's Successors
556 3738
Removing 5098 's Successors
558 3738
562 3738
563 3738
Removing contig-100_1973.1973 's Predecessors
567 3738
568 3738
300
570 3738
Removing 12314 's Successors
574 3738
578 3738
Removing 9529 's Successors
585 3738
587 3738
589 3738
591 3738
Removing contig-100_7220.7220 's Successors
593 3738
594 3738
597 3738
609 3738
Removing 4088 's Predecessors
613 3738
616 3738
618 3738
Removing 2882 's Predecessors
Removing contig-100_8544.63603 's Predecessors
Removing 2440 's Successors
Removing 9436 's Successors
620 3738
623 3738
Removing contig-100_3218.75462 's Predecessors
627 3738
Removing 11566 's Successors
Removing contig-100_814.84200 's Successors
Removing 5286 's Predecessors
628 3738
Removing contig-100_1028.84414 's Successors
629 3738
Removing 11431 's Successors
631 3738
Removing 1314 's Predecessors
Removing 9979 's Successors
633 3738
Removing 11452 's Successors
637 3738
Removing 9403 's Successors
Removing contig-100_

1233 3738
Removing 13206 's Successors
1234 3738
Removing contig-100_2787.79660 's Predecessors
1244 3738
Removing 4815 's Successors
1250 3738
1253 3738
Removing 13292 's Successors
1255 3738
Removing contig-100_4288.69883 's Successors
1260 3738
Removing 13314 's Predecessors
1262 3738
Removing contig-100_2674.79547 's Successors
1263 3738
Removing contig-100_2902.79775 's Predecessors
1265 3738
1267 3738
Removing 8199 's Successors
1268 3738
1269 3738
1272 3738
Removing 1335 's Successors
1273 3738
Removing contig-100_2430.82854 's Predecessors
1278 3738
Removing contig-100_2285.82709 's Predecessors
1280 3738
1287 3738
300
1290 3738
Removing 13424 's Successors
1291 3738
Removing 9856 's Predecessors
1292 3738
Removing 13450 's Successors
1299 3738
1301 3738
Removing contig-100_1600.84986 's Successors
1306 3738
1308 3738
1309 3738
Removing 13494 's Successors
1311 3738
Removing contig-100_1636.88001 's Successors
1313 3738
1316 3738
1318 3738
1325 3738
1326 3738
Removing contig-10

88 3012
Removing 2881 's Successors
89 3012
90 3012
91 3012
Removing contig-100_11692.11692 's Predecessors
Removing 11521 's Successors
Removing 10693 's Successors
92 3012
Removing 12558 's Predecessors
Removing 11815 's Successors
93 3012
Removing 1405 's Successors
Removing 10217 's Successors
95 3012
Removing 4129 's Predecessors
Removing 9939 's Successors
Removing contig-100_574.83393 's Successors
Removing contig-100_10262.64015 's Successors
Removing 12188 's Successors
96 3012
Removing 12892 's Predecessors
Removing 3495 's Predecessors
97 3012
Removing contig-100_5098.70433 's Predecessors
Removing 4713 's Successors
99 3012
100 3012
Removing 4383 's Predecessors
101 3012
102 3012
Removing contig-100_3894.76237 's Successors
103 3012
Removing 9024 's Successors
104 3012
107 3012
108 3012
109 3012
110 3012
Removing 2488 's Predecessors
Removing 2877 's Successors
Removing 5355 's Predecessors
111 3012
114 3012
Removing 1432 's Predecessors
Removing 129 's Predecessors
Removin

533 3012
Removing 2562 's Successors
534 3012
535 3012
Removing 12934 's Predecessors
Removing 12839 's Successors
Removing 1435 's Successors
538 3012
540 3012
544 3012
545 3012
548 3012
300
550 3012
553 3012
555 3012
Removing contig-100_1594.78535 's Successors
556 3012
557 3012
558 3012
Removing 1787 's Predecessors
560 3012
Removing 8214 's Successors
561 3012
563 3012
Removing contig-100_1823.55576 's Successors
564 3012
566 3012
567 3012
568 3012
569 3012
Removing 1176 's Successors
570 3012
Removing 11504 's Predecessors
571 3012
Removing contig-100_1785.78726 's Predecessors
572 3012
575 3012
Removing 4387 's Predecessors
Removing 2520 's Predecessors
577 3012
300
Removing 13949 's Successors
578 3012
Removing 239 's Predecessors
579 3012
300
580 3012
581 3012
582 3012
587 3012
Removing 12738 's Predecessors
588 3012
Removing 4015 's Successors
589 3012
590 3012
Removing 4819 's Successors
591 3012
592 3012
594 3012
597 3012
Removing 11575 's Successors
598 3012
599 3012
600 30

1234 3012
1239 3012
Removing 13742 's Predecessors
1243 3012
Removing contig-100_739.80968 's Predecessors
Removing 8486 's Successors
1245 3012
1247 3012
1250 3012
1252 3012
1255 3012
Removing contig-100_1433.33801 's Predecessors
1256 3012
Removing contig-100_2002.78943 's Successors
1258 3012
1260 3012
1261 3012
1266 3012
Removing contig-100_4918.70253 's Predecessors
Removing 13842 's Successors
1267 3012
1270 3012
1271 3012
300
Removing 13858 's Successors
1272 3012
1273 3012
Removing 1388 's Predecessors
1278 3012
Removing contig-100_2.80231 's Successors
1279 3012
1284 3012
1287 3012
1291 3012
300
1293 3012
1294 3012
Removing contig-100_1411.81640 's Predecessors
1304 3012
Removing contig-100_2929.68264 's Predecessors
1312 3012
Removing 14009 's Successors
1314 3012
Removing contig-100_2806.75149 's Successors
1315 3012
1316 3012
1319 3012
1322 3012
1323 3012
1326 3012
1328 3012
Removing 14057 's Predecessors
1331 3012
1334 3012
300
1336 3012
1340 3012
1344 3012
1348 3012
Remov

In [ ]:
'''for s in samples[4:]:
    sample = s.replace(".txt","")
    if sample[0] == '.':
        continue
    print(s)
    Graph_path= graphspath+sample+'/'+sample+'_scaffolds/oriented.gml'
    G = nx.read_gml(Graph_path)
    df_coverage =Load_Read_Coverage(sample) 
    df_coords, df_coverage_scaffolds = Write_Coverage_Outputs(G, df_coverage)
    df_coords.to_csv(op_path+sample+'_Gbl_Coords.csv')
    df_coverage_scaffolds.to_csv(op_path+sample+'_Coverages.csv')'''